In [14]:
import pandas as pd

import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy import types
import pymysql
pymysql.install_as_MySQLdb()

import os
import sys
sys.path
sys.path.append('../')
sys.path.append('spotify-chart')
from connections import password
import charts

In [15]:
connection_string = (f"root:{password}@localhost")

engine = create_engine(f"mysql://{connection_string}")
engine.execute("DROP DATABASE IF EXISTS spot_db")
engine.execute("CREATE DATABASE spot_db")
engine.execute("ALTER DATABASE spot_db CHARACTER SET = utf8mb4 COLLATE = utf8mb4_unicode_ci")
# engine.execute("ALTER TABLE indeed_resumes CONVERT TO CHARACTER SET utf8mb4 COLLATE utf8mb4_unicode_ci;")
engine = create_engine(f"mysql://{connection_string}/spot_db?charset=utf8mb4")

In [16]:
global_top_200_daily = pd.read_csv('global_top_200_daily.csv')

global_top_200_daily.rename(columns = {'Unnamed: 0':'Position', 
                                'Unnamed: 1': 'Track Name', 
                                'Unnamed: 2':'Artist', 
                                'Note that these figures are generated using a formula that protects against any artificial inflation of chart positions.':'Streams', 
                                'Unnamed: 4':'URL'}, inplace = True)

global_top_200_daily.drop(0, inplace = True)
global_top_200_daily.rename(columns = {'Track Name':'Track'}, inplace = True)

global_top_200_daily['ID'] = global_top_200_daily['URL'].str.split('/', expand = True)[4]

global_top_200_daily.reset_index(inplace = True)

# top 200 db export
global_top_200_db = global_top_200_daily[['Position', 'date', 'ID']]

engine.execute("USE spot_db")
global_top_200_db.to_sql(
    name = 'global_top_200_daily', con = engine,
    if_exists = 'replace', 
    chunksize = 75)
with engine.connect() as con:
    con.execute('ALTER TABLE `global_top_200_daily` ADD PRIMARY KEY (`Index`);')

global_top_200_db.head()

,Position,Position,date,ID
0,1,2,2017-01-01,7BKLCZ1jbUBVqRi2FVlTVw
1,2,3,2017-01-01,4pdPtRcBmOSQDlJ3Fk945m
2,3,4,2017-01-01,5knuzwU65gJK7IF5yJsuaW
3,4,5,2017-01-01,1xznGGDReH1oQq0xzbwXa3
4,5,6,2017-01-01,343YBumqHu19cGoGARUTsd


In [17]:
global_viral_50_daily = pd.read_csv('global_viral_50_daily.csv')

global_viral_50_daily.rename(columns = {'Track Name':'Track'}, inplace = True)

global_viral_50_daily['ID'] = global_viral_50_daily['URL'].str.split('/', expand = True)[4]

global_viral_50_daily.reset_index(inplace = True)

# top 200 db export
global_viral_50_db = global_viral_50_daily[['Position', 'date', 'ID']]

engine.execute("USE spot_db")
global_viral_50_db.to_sql(
    name = 'global_viral_50_daily', con = engine,
    if_exists = 'replace')
with engine.connect() as con:
    con.execute('ALTER TABLE `global_viral_50_daily` ADD PRIMARY KEY (`Index`);')
    
global_viral_50_db.head()

,Position,date,ID
0,1,2017-01-01,0EPxmvsG1BY5td4aTOkWBF
1,2,2017-01-01,4jDmJ51x1o9NZB5Nxxc7gY
2,3,2017-01-01,1D6nV9TPfMnWm7UdVsDVfI
3,4,2017-01-01,0HEmnAUT8PHznIAAmVXqFJ
4,5,2017-01-01,1ZgfAxHQCXLt8o1VXEHHAt


In [18]:
top_200_daily = pd.read_csv('top_200_daily.csv')

top_200_daily.rename(columns = {'Unnamed: 0':'Position', 
                                'Unnamed: 1': 'Track Name', 
                                'Unnamed: 2':'Artist', 
                                'Note that these figures are generated using a formula that protects against any artificial inflation of chart positions.':'Streams', 
                                'Unnamed: 4':'URL'}, inplace = True)
top_200_daily.drop(0, inplace = True)
top_200_daily.rename(columns = {'Track Name':'Track'}, inplace = True)

top_200_daily['ID'] = top_200_daily['URL'].str.split('/', expand = True)[4]

top_200_daily.reset_index(inplace = True)

# top 200 db export
top_200_db = top_200_daily[['Position', 'date', 'ID']]

engine.execute("USE spot_db")
top_200_db.to_sql(
    name = 'top_200_daily', con = engine,
    if_exists = 'replace', 
    chunksize = 75)
with engine.connect() as con:
    con.execute('ALTER TABLE `top_200_daily` ADD PRIMARY KEY (`Index`);')

top_200_db.head()

,Position,Position,date,ID
0,1,2,2017-01-01,343YBumqHu19cGoGARUTsd
1,2,3,2017-01-01,5aAx2yezTd8zXrkmtKl66Z
2,3,4,2017-01-01,7BKLCZ1jbUBVqRi2FVlTVw
3,4,5,2017-01-01,6fujklziTHa8uoM5OQSfIo
4,5,6,2017-01-01,7yyRTcZmCiyzzJlNzGC9Ol


In [19]:
viral_50_daily = pd.read_csv('viral_50_daily.csv')

viral_50_daily.rename(columns = {'Track Name':'Track'}, inplace = True)

viral_50_daily['ID'] = viral_50_daily['URL'].str.split('/', expand = True)[4]

viral_50_daily.reset_index(inplace = True)

# top 200 db export
viral_50_db = viral_50_daily[['Position', 'date', 'ID']]

engine.execute("USE spot_db")
viral_50_db.to_sql(
    name = 'viral_50_daily', con = engine,
    if_exists = 'replace')
with engine.connect() as con:
    con.execute('ALTER TABLE `viral_50_daily` ADD PRIMARY KEY (`Index`);')

viral_50_db.head()

In [21]:
global_unique_viral = global_viral_50_daily.drop_duplicates('ID')[['ID', 'Track', 'Artist']]
global_unique_top = global_top_200_daily.drop_duplicates('ID')[['ID', 'Track', 'Artist']]
unique_viral = viral_50_daily.drop_duplicates('ID')[['ID', 'Track', 'Artist']]
unique_top = top_200_daily.drop_duplicates('ID')[['ID', 'Track', 'Artist']]


unique_songs = pd.concat([unique_viral, unique_top, global_unique_viral, 
                          global_unique_top], ignore_index = True)
unique_songs = unique_songs.drop_duplicates('ID')

engine.execute("USE spot_db")
unique_songs.to_sql(
    name = 'songs', con = engine,
    if_exists = 'replace', 
    dtype={'ID': types.VARCHAR(255)})
with engine.connect() as con:
    con.execute('ALTER TABLE `songs` ADD PRIMARY KEY (`ID`);')

unique_songs.head()

9337